In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import tarfile
import urllib
from scipy import ndimage
import cPickle as pickle
import tensorflow as tf
import matplotlib.image as mpimg

In [2]:
num_classes = 3
image_size = 128  # Pixel width and height.
channel = 3
pixel_depth = 255.0  # Number of levels per pixel.
#folders = ['image_128_v2/label_128_0_v2',
#            'image_128_v2/label_128_1_v2',
#'image_128_v2/label_128_1_v2_clear']
folders = ['py_label_0',
          'py_label_1',
          'py_label_1_clear']

def load(data_folders, min_num_images, max_num_images):
  dataset = np.ndarray(
    shape=(max_num_images, image_size, image_size, channel), dtype=np.float32)
  original_dataset = np.ndarray(
    shape=(max_num_images, image_size, image_size, channel), dtype=np.uint8)
  labels = np.ndarray(shape=(max_num_images), dtype=np.int32)
  label_index = 0
  image_index = 0
  for folder in data_folders:
    print folder
    for image in os.listdir(folder):
      if image_index >= max_num_images:
        raise Exception('More images than expected: %d >= %d' % (
          num_images, max_num_images))
      image_file = os.path.join(folder, image)
      try:
        original_data = mpimg.imread(image_file)
        image_data = (ndimage.imread(image_file).astype(float) -
                      pixel_depth / 2) / pixel_depth
        if image_data.shape != (image_size, image_size,channel):
          raise Exception('Unexpected image shape: %s' % str(image_data.shape))
        dataset[image_index, :, :, :] = image_data
        original_dataset[image_index, :, :, :] = original_data
        labels[image_index] = label_index
        image_index += 1
      except IOError as e:
        print 'Could not read:', image_file, ':', e, '- it\'s ok, skipping.'
    label_index += 1
  num_images = image_index
  dataset = dataset[0:num_images, :, :, :]
  original_dataset = original_dataset[0:num_images, :, :, :]
  labels = labels[0:num_images]
  if num_images < min_num_images:
    raise Exception('Many fewer images than expected: %d < %d' % (
        num_images, min_num_images))
  print 'Full dataset tensor:', dataset.shape
  print 'Full original dataset tensor:', original_dataset.shape
  print 'Mean:', np.mean(dataset)
  print 'Standard deviation:', np.std(dataset)
  print 'Labels:', labels.shape
  return original_dataset,dataset, labels
original_dataset,dataset, labels = load(folders, 10000, 21000)

py_label_0
py_label_1
py_label_1_clear
Full dataset tensor: (20920, 128, 128, 3)
Full original dataset tensor: (20920, 128, 128, 3)
Mean: -0.175889
Standard deviation: 0.234473
Labels: (20920,)


In [3]:
np.random.seed(133)
def randomize(original_dataset,dataset, labels):
  permutation = np.random.permutation(labels.shape[0])
  shuffled_dataset = dataset[permutation,:,:]
  shuffled_original_dataset = original_dataset[permutation,:,:]
  shuffled_labels = labels[permutation]
  return shuffled_original_dataset,shuffled_dataset, shuffled_labels
original_dataset,dataset, labels = randomize(original_dataset,dataset, labels)

In [4]:
train_size = 15920
valid_size = 2500
test_size = 2500
#train_size = 128
#valid_size = 100
#test_size = 20000
valid_dataset = dataset[:valid_size,:,:,:]
valid_labels = labels[:valid_size]
train_dataset = dataset[valid_size:valid_size+train_size,:,:,:]
train_labels = labels[valid_size:valid_size+train_size]
test_dataset = dataset[valid_size+train_size:train_size+valid_size+test_size,:,:,:]
test_labels = labels[valid_size+train_size:train_size+valid_size+test_size]

original_valid_dataset = original_dataset[:valid_size,:,:,:]
original_train_dataset = original_dataset[valid_size:valid_size+train_size,:,:,:]
original_test_dataset = original_dataset[valid_size+train_size:train_size+valid_size+test_size,:,:,:]


print 'Training', train_dataset.shape, train_labels.shape
print 'Validation', valid_dataset.shape, valid_labels.shape
print 'Test', test_dataset.shape, test_labels.shape
print 'original_Training', original_train_dataset.shape
print 'original_Validation', original_valid_dataset.shape
print 'original_Test', original_test_dataset.shape

Training (15920, 128, 128, 3) (15920,)
Validation (2500, 128, 128, 3) (2500,)
Test (2500, 128, 128, 3) (2500,)
original_Training (15920, 128, 128, 3)
original_Validation (2500, 128, 128, 3)
original_Test (2500, 128, 128, 3)


In [5]:
num_labels = 3
num_channels = 3 # grayscale
valid_prediction = np.ndarray(shape=(valid_size, num_labels ), dtype=np.float32)
test_prediction = np.ndarray(shape=(test_size, num_labels ), dtype=np.float32)

import numpy as np

def reformat(dataset, labels):
  dataset = dataset.reshape(
    (-1, image_size, image_size, num_channels)).astype(np.float32)
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print 'Training set', train_dataset.shape, train_labels.shape
print 'Validation set', valid_dataset.shape, valid_labels.shape
print 'Test set', test_dataset.shape, test_labels.shape

Training set (15920, 128, 128, 3) (15920, 3)
Validation set (2500, 128, 128, 3) (2500, 3)
Test set (2500, 128, 128, 3) (2500, 3)


In [6]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])


In [7]:
batch_size = 64
patch_size = 5
depth = 15
num_hidden = 512
lamada = 0.00
calculate_per_times = 10
graph = tf.Graph()

with graph.as_default():

  # Input data.
  tf_train_dataset = tf.placeholder(
    tf.float32, shape=(batch_size, image_size, image_size, num_channels))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))

  #tf_valid_dataset = tf.constant(valid_dataset)
  #tf_test_dataset = tf.constant(test_dataset)

  tf_valid_test_dataset = tf.placeholder(
    tf.float32, shape=(calculate_per_times, image_size, image_size, num_channels))
  
  # Variables.
  conv1_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, num_channels, depth], stddev=0.1))
  conv1_biases = tf.Variable(tf.zeros([depth]))
  conv2_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth, depth*2], stddev=0.1))
  conv2_biases = tf.Variable(tf.constant(0.0, shape=[depth*2]))
  conv3_weights = tf.Variable(tf.truncated_normal(
      [patch_size, patch_size, depth*2, depth*4], stddev=0.1))
  conv3_biases = tf.Variable(tf.constant(0.0, shape=[depth*4]))
  # conv4_weights = tf.Variable(tf.truncated_normal(
  #     [patch_size, patch_size, depth*4, depth*8], stddev=1))
  # conv4_biases = tf.Variable(tf.constant(0.0, shape=[depth*8]))


  full_weights = tf.Variable(tf.truncated_normal(
      [image_size / 8 * image_size / 8 * depth*4, num_hidden], stddev=0.1))
  full_biases = tf.Variable(tf.constant(0.0, shape=[num_hidden]))
  softmax_weights = tf.Variable(tf.truncated_normal(
      [num_hidden, num_labels], stddev=0.1))
  softmax_biases = tf.Variable(tf.constant(0.0, shape=[num_labels]))

  # Dropout.
  keep_prob = tf.placeholder("float")

  #lr = tf.train.exponential_decay(0.001, global_step, 3, 0.5, staircase=True)
  
  # Model.
  def model(data):
    conv_1 = tf.nn.conv2d(data, conv1_weights, [1, 1, 1, 1], padding='SAME')
    relu_1 = tf.nn.relu(conv_1 + conv1_biases)
    max_pool_1 = tf.nn.max_pool(relu_1, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1],padding = 'SAME')
    
    conv_2 = tf.nn.conv2d(max_pool_1, conv2_weights, [1, 1, 1, 1], padding='SAME')
    relu_2 = tf.nn.relu(conv_2 + conv2_biases)
    max_pool_2 = tf.nn.max_pool(relu_2, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1],padding = 'SAME')
    
    conv_3 = tf.nn.conv2d(max_pool_2, conv3_weights, [1, 1, 1, 1], padding='SAME')
    relu_3 = tf.nn.relu(conv_3 + conv3_biases)
    max_pool_3 = tf.nn.max_pool(relu_3, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1],padding = 'SAME')
    
    # conv_4 = tf.nn.conv2d(max_pool_3, conv4_weights, [1, 1, 1, 1], padding='SAME')
    # relu_4 = tf.nn.relu(conv_4 + conv4_biases)
    # max_pool_4 = tf.nn.max_pool(relu_4, ksize = [1, 2, 2, 1], strides = [1, 2, 2, 1],padding = 'SAME')
    
    shape = max_pool_3.get_shape().as_list()
    reshape = tf.reshape(max_pool_3, [shape[0], shape[1] * shape[2] * shape[3]])
    full_layer = tf.nn.relu(tf.matmul(reshape, full_weights) + full_biases)
    softmax_layer = tf.nn.dropout(full_layer, keep_prob)
    return tf.matmul(softmax_layer, softmax_weights) + softmax_biases
  
  # Training computation.
  logits = model(tf_train_dataset)
     
  loss = tf.reduce_mean(
     tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)  + 
      lamada*(tf.nn.l2_loss(conv1_weights) + tf.nn.l2_loss(conv2_weights) +  
          tf.nn.l2_loss(full_weights) + tf.nn.l2_loss(softmax_weights) ))

  optimizer = tf.train.GradientDescentOptimizer(0.005).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)

  valid_test_prediction = tf.nn.softmax(model(tf_valid_test_dataset))
    
  #valid_prediction = tf.nn.softmax(model(tf_valid_dataset))
  #test_prediction = tf.nn.softmax(model(tf_test_dataset))
  saver = tf.train.Saver()

In [15]:
num_steps = 1

with tf.Session(graph=graph) as session:  
  tf.initialize_all_variables().run()
  saver.restore(session,'conv_layer3_128_clear_param7-200000')
  print "Initialized"
  for step in xrange(num_steps):
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    batch_data = train_dataset[offset:(offset + batch_size), :, :, :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels,keep_prob : 0.5}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    #global_step = global_step + 1
    #print(lr.eval())
    if (step % 200 == 0):
      print "Minibatch loss at step", step, ":", l
      print "Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels)
      valid_loop = valid_dataset.shape[0]/calculate_per_times
      for valid_step in xrange(int(valid_loop)):
            valid_start = valid_step * calculate_per_times
            valid_end = (valid_step + 1) * calculate_per_times
            valid_subset = valid_dataset[valid_start:valid_end, :, :, :]
            feed_dict = {tf_valid_test_dataset : valid_subset ,keep_prob : 1}
            valid_prediction[valid_start:valid_end] = session.run(valid_test_prediction, feed_dict = feed_dict)
      print "Validation accuracy: %.1f%%" % accuracy(valid_prediction, valid_labels)
    
      test_loop = test_dataset.shape[0]/calculate_per_times
      for test_step in xrange(int(test_loop)):
            test_start = test_step * calculate_per_times
            test_end = (test_step + 1) * calculate_per_times
            test_subset = test_dataset[test_start:test_end, :, :, :]
            feed_dict = {tf_valid_test_dataset : test_subset ,keep_prob : 1}
            test_prediction[test_start:test_end] = session.run(valid_test_prediction, feed_dict = feed_dict)    
      print "Test accuracy: %.1f%%" % accuracy(test_prediction, test_labels)
    if (step % 10000 == 0):
        saver.save(session,'conv_layer3_128_clear_param7',global_step = step)

Initialized
Minibatch loss at step 0 : 0.00108514
Minibatch accuracy: 100.0%
Validation accuracy: 93.0%
Test accuracy: 90.7%


In [9]:
print test_prediction.shape
i = j = test_prediction.shape[1]
print i,j

(2500, 3)
3 3


In [10]:
def mix_matrix(prediction, labels):
    dim = prediction.shape[1]
    matrix = np.zeros((dim,dim))
    prid_vector = np.argmax(prediction,1)
    labels_vector = np.argmax(labels,1)
    for i in xrange(prid_vector.shape[0]):
        matrix[prid_vector[i]][labels_vector[i]] += 1
    return matrix.astype(int)

In [12]:
matrix = mix_matrix(test_prediction, test_labels)
print matrix

[[1475   55   19]
 [  84  666   52]
 [   5   17  127]]


In [12]:
print np.argmax([[1,2,3]])

2


In [13]:
matrix = mix_matrix(test_prediction, test_labels)
print matrix

[[1475   55   19]
 [  84  666   52]
 [   5   17  127]]


In [14]:
matrix = mix_matrix(valid_prediction, valid_labels)
print matrix

[[1539   64   10]
 [  26  653   61]
 [   3   11  133]]


In [19]:
matrix = mix_matrix(valid_prediction, valid_labels)
print matrix

[[6239   18    3]
 [  18 2925   54]
 [   1   17  725]]


In [15]:
print 126.0/(126 + 18 + 4)  

0.851351351351


In [16]:
import tensorflow

In [40]:
np.argmax?